In [1]:
import cptac 
import pandas as pd
import os, sys
currentdir = os.path.dirname(os.path.realpath('notebook_steps'))
parentdir = os.path.dirname(currentdir)
sys.path.append(parentdir)
import Delta_Correlation as dc

In [2]:
cancer = cptac.Luad()

In [3]:
df = dc.get_prot_trans_df(cancer)
df

cptac warning: Due to dropping the specified levels, dataframe now has 745 duplicated column headers. (C:\Users\nanel\Documents\GitHub\pancancerProteinMRNA\notebook_steps\Delta_Correlation.py, line 15)
cptac warning: Due to dropping the specified levels, dataframe now has 745 duplicated column headers. (C:\Users\nanel\Documents\GitHub\pancancerProteinMRNA\notebook_steps\Delta_Correlation.py, line 25)


,Patient_ID,Gene,Proteomics,Tissue,Transcriptomics
0,C3L-00001,A1BG,-2.5347,tumor,2.2545
1,C3L-00009,A1BG,-0.5627,tumor,1.4770
2,C3L-00080,A1BG,-1.9422,tumor,1.5103
3,C3L-00083,A1BG,2.1636,tumor,3.0398
4,C3L-00093,A1BG,-1.0022,tumor,1.7528
...,...,...,...,...,...
2205156,C3N-02582.N,ZZZ3,0.9497,normal,4.2045
2205157,C3N-02586.N,ZZZ3,1.1807,normal,4.2214
2205158,C3N-02587.N,ZZZ3,1.1825,normal,4.0836
2205159,C3N-02588.N,ZZZ3,1.2729,normal,4.3164


In [4]:
test_genes = pd.unique(df.Gene)[0:5]

In [5]:
test_df = df[df.Gene.isin(test_genes)]
test_df

,Patient_ID,Gene,Proteomics,Tissue,Transcriptomics
0,C3L-00001,A1BG,-2.5347,tumor,2.2545
1,C3L-00009,A1BG,-0.5627,tumor,1.4770
2,C3L-00080,A1BG,-1.9422,tumor,1.5103
3,C3L-00083,A1BG,2.1636,tumor,3.0398
4,C3L-00093,A1BG,-1.0022,tumor,1.7528
...,...,...,...,...,...
1150110,C3N-02582.N,AADAC,0.6852,normal,2.1092
1150111,C3N-02586.N,AADAC,1.4085,normal,1.0860
1150112,C3N-02587.N,AADAC,4.8264,normal,3.3245
1150113,C3N-02588.N,AADAC,4.5153,normal,3.5570


In [6]:
import time

In [7]:
start = time.time()
results = test_df.groupby('Gene').apply(dc.regression)
print(time.time() - start)

13.386112451553345


In [8]:
results

Gene
A1BG     {'interaction_coeff': -0.005621377742532035, '...
A2M      {'interaction_coeff': -0.7324729815685269, 'co...
AAAS     {'interaction_coeff': 0.3098589828045592, 'con...
AACS     {'interaction_coeff': 1.7600473881731062, 'con...
AADAC    {'interaction_coeff': -0.7163769385780298, 'co...
dtype: object

In [21]:
reg_df = pd.DataFrame(list(results))
reg_df.index = results.index
reg_df.reset_index(inplace = True)
reg_df = reg_df.dropna()
reg_df['interaction_FDR'] = ssm.fdrcorrection(reg_df['interaction_pval'])[1]
reg_df['condition_FDR'] = ssm.fdrcorrection(reg_df['condition_pval'])[1]
reg_df['intercept_FDR'] = ssm.fdrcorrection(reg_df['intercept_pval'])[1]
reg_df

,Gene,interaction_coeff,condition_coeff,transcript_coeff,intercept,interaction_pval,condition_pval,transcript_pval,intercept_pval,interaction_FDR,condition_FDR,intercept_FDR
0,A1BG,-0.005621,-3.138192,-0.069847,1.982755,0.983767,6.692320e-14,7.854612e-01,6.670211e-08,0.983767,3.346160e-13,1.667553e-07
1,A2M,-0.732473,4.320107,1.371296,-11.647115,0.128614,3.732341e-01,2.917195e-03,1.347250e-02,0.214357,3.779691e-01,1.684063e-02
2,AAAS,0.309859,-1.068018,0.077440,-0.282065,0.268019,3.044971e-01,7.513657e-01,7.524439e-01,0.335023,3.779691e-01,7.524439e-01
3,AACS,1.760047,-3.663687,0.273041,-2.546285,0.000001,1.523813e-04,3.992679e-01,2.589706e-03,0.000005,3.809533e-04,4.316177e-03
4,AADAC,-0.716377,0.332234,1.616517,-1.877289,0.000211,3.779691e-01,9.359110e-18,3.903905e-08,0.000527,3.779691e-01,1.667553e-07


In [15]:
reg_df['interaction_FDR'] = ssm.fdrcorrection(reg_df['interaction_pval'])
reg_df['condition_FDR'] = ssm.fdrcorrection(reg_df['condition_pval'])
reg_df['intercept_FDR'] = ssm.fdrcorrection(reg_df['intercept_pval'])
reg_df

,interaction_coeff,condition_coeff,transcript_coeff,intercept,interaction_pval,condition_pval,transcript_pval,intercept_pval
Gene,,,,,,,,
A1BG,-0.005621,-3.138192,-0.069847,1.982755,0.983767,6.692320e-14,7.854612e-01,6.670211e-08
A2M,-0.732473,4.320107,1.371296,-11.647115,0.128614,3.732341e-01,2.917195e-03,1.347250e-02
AAAS,0.309859,-1.068018,0.077440,-0.282065,0.268019,3.044971e-01,7.513657e-01,7.524439e-01
AACS,1.760047,-3.663687,0.273041,-2.546285,0.000001,1.523813e-04,3.992679e-01,2.589706e-03
AADAC,-0.716377,0.332234,1.616517,-1.877289,0.000211,3.779691e-01,9.359110e-18,3.903905e-08


In [13]:
list(results)

[{'interaction_coeff': -0.005621377742532035,
  'condition_coeff': -3.138191751761857,
  'transcript_coeff': -0.0698474956997143,
  'intercept': 1.9827545815127643,
  'interaction_pval': 0.9837666980841135,
  'condition_pval': 6.692320347219538e-14,
  'transcript_pval': 0.7854612291907325,
  'intercept_pval': 6.67021134035188e-08},
 {'interaction_coeff': -0.7324729815685269,
  'condition_coeff': 4.320106808375722,
  'transcript_coeff': 1.3712963041007216,
  'intercept': -11.64711542661222,
  'interaction_pval': 0.12861448532769265,
  'condition_pval': 0.3732341271332117,
  'transcript_pval': 0.0029171947230322115,
  'intercept_pval': 0.013472500900094433},
 {'interaction_coeff': 0.3098589828045592,
  'condition_coeff': -1.0680182194239456,
  'transcript_coeff': 0.07744015339345421,
  'intercept': -0.282064610526283,
  'interaction_pval': 0.2680186000391487,
  'condition_pval': 0.304497050482342,
  'transcript_pval': 0.7513657495382859,
  'intercept_pval': 0.752443948475882},
 {'interac

In [18]:
import statsmodels.stats.multitest as ssm